# SYMBOLIC_SENSITIVITY

## Overview
Automatically computes how the outputs of a model change with respect to changes in parameters or initial conditions. This function provides a sensitivity matrix or vector, enabling business users to assess the impact of parameter variations on model results directly in Excel. It leverages the CasADi package for symbolic and algorithmic differentiation, and is accessible via the Boardflare Python for Excel add-in.

From a technical perspective, the function uses symbolic differentiation to compute the Jacobian matrix of the model output with respect to the specified parameters. Given a model $f(\mathbf{x}, \mathbf{a})$ where $\mathbf{x}$ are variables and $\mathbf{a}$ are parameters, the sensitivity matrix $S$ is defined as:

```math
S = \frac{\partial f}{\partial \mathbf{a}}
```

This means each entry $S_{ij}$ represents the partial derivative of the $i$-th output with respect to the $j$-th parameter, evaluated at the provided values. The CasADi library constructs the symbolic computation graph and efficiently evaluates these derivatives using algorithmic differentiation, ensuring both accuracy and performance for complex models.

## Usage
To use the `SYMBOLIC_SENSITIVITY` function in Excel, enter it as a formula in a cell, specifying your model, variables, parameters, and their names:

```excel
=SYMBOLIC_SENSITIVITY(model, variables, parameters, [variable_names], [parameter_names])
```

## Arguments
| Argument        | Type           | Required | Description                                                                 | Example                       |
|-----------------|----------------|----------|-----------------------------------------------------------------------------|-------------------------------|
| model           | string         | Yes      | Symbolic expression for the model output as a function of variables and parameters. | "x**2 + a*y"           |
| variables       | 2D list float  | Yes      | Values for the model's variables at which to compute sensitivities.         | [[1.0, 2.0]]                  |
| parameters      | 2D list float  | Yes      | Values for the model's parameters at which to compute sensitivities.        | [[0.5, 1.5]]                  |
| variable_names  | 2D list string | No       | Names of the variables (if not inferred from the model).                    | [["x", "y"]]                |
| parameter_names | 2D list string | No       | Names of the parameters (if not inferred from the model).                   | [["a"]]                      |

## Returns
| Type         | Description                                                      | Example         |
|--------------|------------------------------------------------------------------|-----------------|
| list[list[float]]| Sensitivity matrix or vector showing how outputs change with respect to parameters. | [[2.0, 1.0]]    |
| str          | Error message if the calculation fails or input is invalid.       | "Invalid input: model must be a valid expression." |

## Limitations
- The model must be a valid Python expression using the provided variable and parameter names.
- Only supports scalar or 2D list arguments of type float or string.
- Sensitivity analysis is local and only valid near the evaluation point.

## Benefits
Excel does not provide built-in tools for symbolic or algorithmic sensitivity analysis. This Python function, powered by CasADi, enables advanced model analysis directly in Excel, allowing users to quickly assess the impact of parameter changes without manual differentiation or simulation.

## Examples

### Financial Forecasting
*Business context*: A financial analyst wants to understand how changes in interest rate and growth rate parameters affect projected revenue.

Excel usage:
```excel
=SYMBOLIC_SENSITIVITY("x*a + y*b", {100, 200}, {0.05, 0.03}, {"x", "y"}, {"a", "b"})
```
*Expected outcome*: Returns the sensitivity of revenue to changes in interest rate and growth rate.

### Engineering Process Control
*Business context*: An engineer analyzes how variations in process parameters affect system output.

Excel usage:
```excel
=SYMBOLIC_SENSITIVITY("x**2 + a*y", {2, 3}, {1.5}, {"x", "y"}, {"a"})
```
*Expected outcome*: Returns the sensitivity of the output to the parameter 'a' at the given point.

In [ ]:
import casadi as ca

def symbolic_sensitivity(model, variables, parameters, variable_names=None, parameter_names=None):
    """
    Computes the sensitivity matrix of a model output with respect to parameters using CasADi.

    Args:
        model: str, symbolic expression for the model output (e.g., "x**2 + a*y").
        variables: 2D list of floats, values for the model's variables (e.g., [[1.0, 2.0]]).
        parameters: 2D list of floats, values for the model's parameters (e.g., [[0.5, 1.5]]).
        variable_names: 2D list of strings (optional), names of the variables (e.g., [["x", "y"]]).
        parameter_names: 2D list of strings (optional), names of the parameters (e.g., [["a"]]).

    Returns:
        list[list[float]]: Sensitivity matrix or vector, or error message string.
    """
    try:
        if not isinstance(model, str):
            return "model must be a string."
        if not (isinstance(variables, list) and len(variables) > 0 and isinstance(variables[0], list)):
            return "variables must be a 2D list of floats."
        if not (isinstance(parameters, list) and len(parameters) > 0 and isinstance(parameters[0], list)):
            return "parameters must be a 2D list of floats."
        var_vals = variables[0]
        param_vals = parameters[0]
        if variable_names is not None:
            if not (isinstance(variable_names, list) and len(variable_names) > 0 and isinstance(variable_names[0], list)):
                return "variable_names must be a 2D list of strings."
            var_names = variable_names[0]
        else:
            var_names = [f"x{i+1}" for i in range(len(var_vals))]
        if parameter_names is not None:
            if not (isinstance(parameter_names, list) and len(parameter_names) > 0 and isinstance(parameter_names[0], list)):
                return "parameter_names must be a 2D list of strings."
            param_names = parameter_names[0]
        else:
            param_names = [f"a{i+1}" for i in range(len(param_vals))]
        if len(var_names) != len(var_vals):
            return "Number of variable names and values must match."
        if len(param_names) != len(param_vals):
            return "Number of parameter names and values must match."
        sym_vars = [ca.MX.sym(name) for name in var_names]
        sym_params = [ca.MX.sym(name) for name in param_names]
        vars_dict = {name: sym_vars[i] for i, name in enumerate(var_names)}
        params_dict = {name: sym_params[i] for i, name in enumerate(param_names)}
        try:
            expr = eval(model, {**vars_dict, **params_dict, 'ca': ca})
        except Exception as e:
            return f"Invalid model expression: {str(e)}"
        S = ca.jacobian(expr, ca.vertcat(*sym_params))
        sens_func = ca.Function('sens_func', sym_vars + sym_params, [S])
        result_matrix = sens_func(*(var_vals + param_vals))
        if isinstance(result_matrix, ca.DM):
            return result_matrix.full().tolist()
        else:
            return "Error during CasADi calculation: Unexpected result type."
    except ca.CasadiException as e:
        return f"Error during CasADi calculation: {e}"
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_demo_finance():
    result = symbolic_sensitivity(
        "x*a + y*b",
        [[100, 200]],
        [[0.05, 0.03]],
        [["x", "y"]],
        [["a", "b"]]
    )
    assert isinstance(result, list)
    assert len(result) == 1
    assert all(isinstance(x, float) or isinstance(x, int) for x in result[0])

def test_demo_engineering():
    result = symbolic_sensitivity(
        "x**2 + a*y",
        [[2, 3]],
        [[1.5]],
        [["x", "y"]],
        [["a"]]
    )
    assert isinstance(result, list)
    assert len(result) == 1
    assert all(isinstance(x, float) or isinstance(x, int) for x in result[0])

def test_error_invalid_model():
    result = symbolic_sensitivity(
        "x**2 +",
        [[2, 3]],
        [[1.5]],
        [["x", "y"]],
        [["a"]]
    )
    assert isinstance(result, str) and ("error" in result.lower() or "invalid" in result.lower())

def test_error_mismatched_names_and_values():
    result = symbolic_sensitivity(
        "x*a + y*b",
        [[100, 200]],
        [[0.05]],
        [["x", "y"]],
        [["a", "b"]]
    )
    assert isinstance(result, str) and ("must match" in result.lower() or "must be" in result.lower())

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    [
        "x*a + y*b",
        [[100, 200]],
        [[0.05, 0.03]],
        [["x", "y"]],
        [["a", "b"]]
    ],
    [
        "x**2 + a*y",
        [[2, 3]],
        [[1.5]],
        [["x", "y"]],
        [["a"]]
    ]
]

demo = gr.Interface(
    fn=symbolic_sensitivity,
    inputs=[
        gr.Textbox(label="Model Expression (e.g. x*a + y*b)", lines=2, value="x*a + y*b"),
        gr.Dataframe(label="Variables (row: [x, y, ...])", headers=None, datatype="number", row_count=(1, "fixed"), col_count=(2, "fixed"), type="array", value=[[100, 200]]),
        gr.Dataframe(label="Parameters (row: [a, b, ...])", headers=None, datatype="number", row_count=(1, "fixed"), col_count=(2, "fixed"), type="array", value=[[0.05, 0.03]]),
        gr.Dataframe(label='Variable Names (row: ["x", "y", ...])', headers=None, datatype="str", row_count=(1, "fixed"), col_count=(2, "fixed"), type="array", value=[["x", "y"]]),
        gr.Dataframe(label='Parameter Names (row: ["a", "b", ...])', headers=None, datatype="str", row_count=(1, "fixed"), col_count=(2, "fixed"), type="array", value=[["a", "b"]]),
    ],
    outputs=gr.Dataframe(label="Sensitivity Matrix or Error Message"),
    examples=examples,
    description="Calculate the symbolic sensitivity matrix for a model with respect to its parameters.",
    flagging_mode="never",
)
demo.launch()